# Algoritmo de Asignación de Tareas

In [17]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment

In [18]:
def generar_costos_aleatorios(num_trabajadores, num_tareas):
    costos = np.random.randint(1, 101, size=(num_trabajadores, num_tareas))
    columnas = [f"Tarea {i+1}" for i in range(num_tareas)]
    filas = [f"Trabajador {i+1}" for i in range(num_trabajadores)]
    df = pd.DataFrame(costos, index=filas, columns=columnas)
    print("\nMatriz de costos generada aleatoriamente:")
    print(df)
    return costos

In [19]:
def ingresar_costos_manualmente(num_trabajadores, num_tareas):
    print(f"\nIntroduce los costos de asignación para {num_trabajadores} trabajadores y {num_tareas} tareas:")
    matriz = []
    for i in range(num_trabajadores):
        fila = []
        for j in range(num_tareas):
            while True:
                try:
                    valor = int(input(f"Costo para Trabajador {i+1}, Tarea {j+1}: "))
                    fila.append(valor)
                    break
                except ValueError:
                    print("Por favor, introduce un número entero.")
        matriz.append(fila)
    
    costos = np.array(matriz)
    columnas = [f"Tarea {i+1}" for i in range(num_tareas)]
    filas = [f"Trabajador {i+1}" for i in range(num_trabajadores)]
    df = pd.DataFrame(costos, index=filas, columns=columnas)
    print("\nMatriz de costos ingresada manualmente:")
    print(df)
    return costos

In [20]:
def rellenar_matriz(costos):
    filas, columnas = costos.shape
    if filas == columnas:
        return costos
    if filas > columnas:
        padding = np.full((filas, filas - columnas), 9999)
        costos = np.hstack((costos, padding))
    else:
        padding = np.full((columnas - filas, columnas), 9999)
        costos = np.vstack((costos, padding))
    return costos

In [21]:
def algoritmo_asignacion(costos):
    fila_indices, columna_indices = linear_sum_assignment(costos)
    asignaciones = list(zip(fila_indices, columna_indices))
    costo_total = costos[fila_indices, columna_indices].sum()
    return asignaciones, costo_total

In [22]:
def hungarian_algorithm(cost_matrix):
    cost_matrix = cost_matrix.copy()
    n = cost_matrix.shape[0]

    print("Paso 1: reducción por filas")
    for i in range(n):
        cost_matrix[i] -= cost_matrix[i].min()
        print(cost_matrix[i])

    print("\nPaso 2: reducción por columnas")
    for j in range(n):
        cost_matrix[:, j] -= cost_matrix[:, j].min()
        print(cost_matrix[:, j])

    # encuentra ceros únicos
    def find_zeroes(m):
        return [(i, j) for i in range(n) for j in range(n) if m[i][j] == 0]

    # encuentra todas las coordenadas donde hay ceros en la matriz
    # este asigna sin que dos estén en la misma fila o columna
    def cover_zeroes(m):
        row_covered = [False] * n
        col_covered = [False] * n
        zero_locations = find_zeroes(m)
        print(f"\nCeros encontrados: {zero_locations}")
        marked_zeros = []

        for r, c in zero_locations:
            if not row_covered[r] and not col_covered[c]:
                marked_zeros.append((r, c))
                row_covered[r] = True
                col_covered[c] = True
                # print(f" fila - {r}: {row_covered}")
                # print(f" colu - {c}: {col_covered}")

        return marked_zeros

    def min_uncovered(m, row_cov, col_cov):
        return min(
            [m[i][j] for i in range(n) for j in range(n)
             if not row_cov[i] and not col_cov[j]]
        )

    while True:
        marked = cover_zeroes(cost_matrix)
        print(f"Combinacion: {marked}")
        # si trabajadores == tareas
        if len(marked) == n:
            return marked  # lista de asignaciones (trabajador, tarea)
        else:
            print(f"  Opcion no cubre asignacion de todas las tareas o trabajadores")

        # si trabajadores != tareas
        row_cov = [False] * n
        col_cov = [False] * n
        for r, c in marked:
            col_cov[c] = True

        # por si hay ceros sin cubrir
        while True:
            zero_found = False
            for i in range(n):
                for j in range(n):
                    if cost_matrix[i][j] == 0 and not row_cov[i] and not col_cov[j]:
                        row_cov[i] = True
                        col_cov[j] = False
                        zero_found = True
            if not zero_found:
                break
        
        # se crean nuevos ceros si comb actual no es la correcta
        min_val = min_uncovered(cost_matrix, row_cov, col_cov)
        for i in range(n):
            for j in range(n):
                if not row_cov[i] and not col_cov[j]:
                    # resta ese valor de las posiciones no cubiertas (crea más ceros)
                    cost_matrix[i][j] -= min_val
                if row_cov[i] and col_cov[j]:
                    # suma ese valor a las posiciones doblemente cubiertas (para mantener el equilibrio)
                    cost_matrix[i][j] += min_val
        
        print("\nMatriz ajustada:")
        print(cost_matrix)
        print("Ceros después del ajuste:", find_zeroes(cost_matrix))

In [23]:
def algoritmo_asignacion_manual(costos):
    asignaciones = hungarian_algorithm(costos)
    costo_total = sum(costos[i][j] for i, j in asignaciones)
    return asignaciones, costo_total

In [24]:
print("Algoritmo de asignación de tareas\n")
num_trabajadores = int(input("Número de trabajadores: "))
num_tareas = int(input("Número de tareas: "))


Algoritmo de asignación de tareas



### Elegir el método

In [25]:
print("\nOpciones:")
print("1. Ingresar manualmente los costos")
print("2. Generar matriz aleatoriamente")

datos = input("Selecciona una opción (1 o 2): ")
if datos == "1":
    costos = ingresar_costos_manualmente(num_trabajadores, num_tareas)
elif datos == "2":
    costos = generar_costos_aleatorios(num_trabajadores, num_tareas)
else:
    print("Opción inválida. Elige 1 o 2.")

matriz_completa = rellenar_matriz(costos)


Opciones:
1. Ingresar manualmente los costos
2. Generar matriz aleatoriamente

Matriz de costos generada aleatoriamente:
               Tarea 1  Tarea 2  Tarea 3  Tarea 4  Tarea 5  Tarea 6  Tarea 7  \
Trabajador 1         4       23       86       28       31       60       33   
Trabajador 2         1       37        3       96       19       46       38   
Trabajador 3        57       74       31       40       77       10       33   
Trabajador 4        96       18       97       83       69       61       79   
Trabajador 5        15       95       71       63       14       51       41   
Trabajador 6        91        1       37       69       61       62       90   
Trabajador 7         2       83       78       23       42       97       58   
Trabajador 8        46       30       31       42       46        6       95   
Trabajador 9         5       15       60       34       42       23       54   
Trabajador 10       19       91       99       42       37       60       45  

In [26]:
opt = int(input("Ingrese la opcion que quiere intentar:\n 1. Algoritmo de asignacion de Python\n 2. Algoritmo de asignacion manual"))
if opt == 2:
    asignaciones, costo_total = algoritmo_asignacion_manual(matriz_completa)
else:
    asignaciones, costo_total = algoritmo_asignacion(matriz_completa)

Paso 1: reducción por filas
[ 0 19 82 24 27 56 29 36 86 85]
[ 0 36  2 95 18 45 37 65 38 60]
[47 64 21 30 67  0 23 73  0 49]
[94 16 95 81 67 59 77 71  0 82]
[ 1 81 57 49  0 37 27 55  9 62]
[90  0 36 68 60 61 89  6 92 54]
[ 0 81 76 21 40 95 56  1 60 16]
[40 24 25 36 40  0 89 20 47 17]
[ 0 10 55 29 37 18 49 11 10 91]
[ 0 72 80 23 18 41 26 10 72 61]

Paso 2: reducción por columnas
[ 0  0 47 94  1 90  0 40  0  0]
[19 36 64 16 81  0 81 24 10 72]
[80  0 19 93 55 34 74 23 53 78]
[ 3 74  9 60 28 47  0 15  8  2]
[27 18 67 67  0 60 40 40 37 18]
[56 45  0 59 37 61 95  0 18 41]
[ 6 14  0 54  4 66 33 66 26  3]
[35 64 72 70 54  5  0 19 10  9]
[86 38  0  0  9 92 60 47 10 72]
[69 44 33 66 46 38  0  1 75 45]

Ceros encontrados: [(0, 0), (1, 0), (1, 2), (2, 5), (2, 6), (2, 8), (3, 8), (4, 4), (5, 1), (6, 0), (6, 3), (6, 7), (6, 9), (7, 5), (8, 0), (9, 0)]
Combinacion: [(0, 0), (1, 2), (2, 5), (3, 8), (4, 4), (5, 1), (6, 3)]
  Opcion no cubre asignacion de todas las tareas o trabajadores

Matriz ajustada:

In [27]:
print("\nAsignaciones óptimas:")
for trabajador, tarea in asignaciones:
    if trabajador < num_trabajadores and tarea < num_tareas:
        print(f"Trabajador {trabajador + 1} → Tarea {tarea + 1} (Costo: {costos[trabajador][tarea]})")
print(f"\nCosto total mínimo: {costo_total}")


Asignaciones óptimas:
Trabajador 1 → Tarea 4 (Costo: 28)
Trabajador 2 → Tarea 3 (Costo: 3)
Trabajador 3 → Tarea 6 (Costo: 10)
Trabajador 4 → Tarea 9 (Costo: 2)
Trabajador 5 → Tarea 5 (Costo: 14)
Trabajador 6 → Tarea 2 (Costo: 1)
Trabajador 7 → Tarea 8 (Costo: 3)
Trabajador 8 → Tarea 10 (Costo: 23)
Trabajador 9 → Tarea 1 (Costo: 5)
Trabajador 10 → Tarea 7 (Costo: 45)

Costo total mínimo: 134
